In [ ]:
# load libraries and scripts
include("../scripts/tools.jl")
include("../scripts/get_grid.jl")
include("../scripts/get_params.jl")
include("../scripts/stable.jl")
include("../scripts/dynamics.jl")
include("../scripts/disturbances.jl")
include("../scripts/plotting.jl")
include("../scripts/ps_analysis.jl");

In [ ]:
#load borders
border, scale_factor = import_border("../data/borders/border.json")

dx = 0.005
grid = get_grid(border, dx);

In [ ]:
#scatter(grid.coord[grid.isgrid,2],grid.coord[grid.isgrid,1])
#scatter!(grid.coord[grid.isborder,2],grid.coord[grid.isborder,1])

In [ ]:
# compute or load grid parameters
contmod = get_params(grid, scale_factor, "../data/disc/pantagruel.h5", Niter=10,
    dmax = 2*dx, patch=4000.0, bmin=0.0, tau=0.00001);

In [ ]:
heatmap(reshape(contmod.bx, contmod.Ny, contmod.Nx))

In [ ]:
compute_stable_sol!(contmod, interval = 10000, Niter = 60000, tol = 1e-7);

In [ ]:
dm = load_discrete_model("../data/disc/pantagruel.h5", scale_factor)
#th = set_ref_phase!(contmod, vec(dm.coord[1,:]), th_ref = 0.0)
th_disc = get_discrete_values(contmod.coord[contmod.isgrid,:], dm.coord, contmod.th[contmod.isgrid]);

In [ ]:
scatter(dm.th, th_disc)
th_min = minimum(dm.th)
th_max = maximum(dm.th)
plot!([th_min; th_max], [th_min; th_max], xlabel="\$\\theta_i^{\\rm ref}\$",
    ylabel="\$\\theta_i\$",color=:red, lw = 2, legend=false, size=(400,400), xlim=(-1.6,1.6), ylim=(-1.6, 1.6))
savefig("../numerics/theta_ref_vs_cont.pdf")

In [ ]:
#thr = get_cont_values(isgrid, coord[isgrid,:], coord_disc, th_ref);

In [ ]:
#heatmap(reshape(abs.(thr-th), Ny, Nx))

In [ ]:
disc_plot(dm.coord, dm.th, borders=[border])
savefig("../numerics/disc_th.pdf")
#plot!(colorbar=true)
#plot!(clims=(0.0,1.0), c=:viridis, colorbar_title="cbar", grid=false)

In [ ]:
hm_plot(contmod, contmod.th, cb_title = "\$\\theta(x)\$", borders=[border])
#hm_plot(contmod, contmod.th, cb_title = "\$\\theta(x)\$")
savefig("../numerics/cont_theta.svg")
#plot!(clims=(0.0,1.0), c=:viridis, colorbar_title="cbar", grid=false)

In [ ]:
add_local_disturbance!(contmod, [-0.35; 0.24], -10.0, 0.02, dx);

In [ ]:
ts, thetas, omegas = perform_dyn_sim(contmod, interval=100,
    Ndt=5000, dt=0.01, method="crank-nicolson");

In [ ]:
sum(contmod.p)

In [ ]:
#hm_movie(contmod, ts, omegas, tstart = 0.0, tend = 20.0, interval = 1)
c = [500 -250; 0.0 0.0; -500 500; 500. 500.; -1500.0 -1000.0; 750.0 -1000; -1000 -400] / scale_factor
time_plot(contmod, ts, omegas, c, borders = [border], tstart = 0.0, tend = 50.0)

In [ ]:
hm_plot(contmod, contmod.th, cb_title = "\$\\theta(x)\$", borders=[border])

In [ ]:
println(size(thetas))
println(sum(contmod.isgrid))
println(size(contmod.th))

In [ ]:
temp = zeros(size(contmod.th))
temp[contmod.isgrid] = thetas[:,end]

hm_plot(contmod, abs.(contmod.th-temp), cb_title = "\$\\theta(x)\$", borders=[border])